In [5]:
import os, json, pickle, pprint, time
from SPARQLWrapper import SPARQLWrapper, JSON

In [8]:
output = json.load(open('/data/cm/data/entity_classification/entity_classification.json', 'r', encoding='utf-8'))
processed = len([ent['p'] for ent in output.values() if ent['p'] is True])
print("Processed {} entities of {}.".format(processed, len(output)))

Processed 153332 entities of 153332.


In [11]:
print(len(output))

153332


In [7]:
i = 0
for entity in output.keys():
    if output[entity]['p'] is False:
        #print("|", end="")
        #print(entity, "TODO", end=" ")
        resource = 'http://de.dbpedia.org/resource/'+entity
        # perform query
        sparql = SPARQLWrapper("http://de.dbpedia.org/sparql")
        sparql.setQuery("""
            SELECT DISTINCT ?entity ?entitytype
            WHERE {
                ?entity a ?entitytype .
                FILTER (?entity = <"""+resource+""">)
                }

        """)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        #print(len(results['results']['bindings']))
        for res in results['results']['bindings']:
            entitytype = res['entitytype']['value']
            #print(entitytype)
            output[entity]['cats'].append(entitytype)
        output[entity]['p'] = True
        time.sleep(3.0)
        i += 1
    
    
    if i != 0 and i % 2 == 0:
        json.dump(output, open('/data/cm/data/entity_classification/entity_classification.json', 'w', encoding="utf-8"), indent=2, ensure_ascii=False)
        output = json.load(open('/data/cm/data/entity_classification/entity_classification.json', 'r', encoding="utf-8"))
        print(i, end=" ")

2 4 6 8 